In [1]:
# core libraries
import pickle
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
from datetime import datetime


from queries import final_features
from score_funcs import Score
sc = Score()


/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [2]:
os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


#### Data pull for scoring 

In [3]:
querySno = SQLQuery('snowflake')
df = querySno(final_features)

In [4]:
df.head()

,business_id,due_date,emi_no,median_running_balance_2nd_m,median_amount_debited_2m,distinct_debit_txns_1m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_amt_drawn_mom,ratio_ach_debit_credit_freq_1m,ratio_amt_cashin_mom,ratio_revenue_1m_2m,distinct_cashins_1m_prev
0,e48c2153-6bec-4711-92b6-04f1fcbf8b62,2022-12-23,1,114.89,2.0,42,1.305130,0.736842,0.000000,0.000000,0.000000,0.177905,0.0
1,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-24,1,57.47,0.0,14,0.621232,0.062500,0.089158,0.000000,1.979837,0.043739,7.5
2,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-29,2,77.47,0.0,9,0.621232,0.043478,0.089158,0.000000,1.979837,0.043739,7.5
3,aff2ee7d-3db8-4952-b153-cfa11ee2ea0f,2022-12-31,1,2930.33,25.0,46,1.046365,0.500000,0.612804,0.600000,1966.660000,0.870828,1.0
4,04b9db4e-162b-41c4-8434-2187cab6e927,2022-12-31,1,4235.39,25.0,47,1.353830,0.480000,0.000000,0.461538,806.000000,0.996367,1.0


In [5]:
df.shape

(5725, 13)

In [6]:
data_params = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/data_params_v2.pkl')
model = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/redec_3.pkl')
transformer = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/scale_train_object.pkl')

In [ ]:
# df.isnull().sum()

In [7]:
sc.features

['distinct_debit_txns_1m',
 'median_amount_debited_2m',
 'ratio_debit_credit_2m',
 'ratio_ach_credit_freq_total_credit_1m',
 'ratio_ach_debit_credit_freq_1m',
 'median_running_balance_2nd_m',
 'ratio_revenue_1m_2m',
 'distinct_cashins_1m_prev',
 'ratio_amt_drawn_mom',
 'ratio_amt_cashin_mom']

In [8]:
df['emi_no'].value_counts()

1     2280
2     1713
3     1167
4      363
5      197
6        1
7        1
8        1
9        1
10       1
Name: emi_no, dtype: int64

In [ ]:
# df = df[df['emi_no'].isin([2,3,4])]

In [9]:
df0 = sc.preprocess(df,sc.features,data_params,transformer)

In [10]:
df0 = sc.model_scoring_redec(df0,sc.features,model)

In [11]:
df0.head()

,business_id,due_date,emi_no,median_running_balance_2nd_m,median_amount_debited_2m,distinct_debit_txns_1m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_amt_drawn_mom,ratio_ach_debit_credit_freq_1m,ratio_amt_cashin_mom,ratio_revenue_1m_2m,distinct_cashins_1m_prev,redec_proba,redec_score,redec_bin
0,e48c2153-6bec-4711-92b6-04f1fcbf8b62,2022-12-23,1,-0.551024,-0.505696,-0.280372,0.457713,0.805573,-0.275490,-0.732551,-0.350400,-0.155074,-1.054347,0.568455,431.545022,2
1,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-24,1,-0.563283,-0.554756,-0.498307,-1.195758,-1.656611,-0.275416,-0.732551,-0.347744,-0.156665,2.881640,0.579811,420.188937,2
2,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-29,2,-0.559013,-0.554756,-0.537224,-1.195758,-1.726065,-0.275416,-0.732551,-0.347744,-0.156665,2.881640,0.578490,421.509705,2
3,aff2ee7d-3db8-4952-b153-cfa11ee2ea0f,2022-12-31,1,0.050053,0.058486,-0.249238,-0.167908,-0.059194,-0.274980,-0.393512,2.288304,-0.146859,-0.529549,0.448140,551.860057,3
4,04b9db4e-162b-41c4-8434-2187cab6e927,2022-12-31,1,0.328674,0.058486,-0.241455,0.575456,-0.132219,-0.275490,-0.471752,0.731025,-0.145371,-0.529549,0.395666,604.333794,3


In [16]:
# df0 = df0[df0['emi_no'].isin([2,3,4])]
# df0.shape

(3243, 16)

In [12]:
ddf = pd.read_pickle(f'/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/new_DPD_dataset.pkl')

In [13]:
ddf.shape

(5314, 12)

In [14]:
ddf.head()

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no,DPD_plus_15_next,rs2_bin,rs2_score,fico_score,rs1_score,rs1_bin
0,979c67cc-8eac-42f2-a507-e02eb211b1a7,2023-05-04,0,0,0,1,0.0,3,576.0,706,829.7607925,5
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,2023-05-04,0,0,0,1,0.0,5,850.0,846,983.9164071,5
2,366f04c0-96de-466e-a500-ef43e7de7ab1,2023-05-04,0,0,0,1,0.0,5,810.0,693,904.1735203,5
3,0e04e84c-edcc-4809-92b3-4e8ba5a8c7e5,2023-05-04,0,0,0,1,0.0,5,998.0,737,925.3730837,5
4,dffa16f0-15c0-475f-b391-29726ee1e2f3,2023-05-04,0,0,0,1,0.0,4,663.0,674,769.8091898,5


In [17]:
# df0 = df0[['business_id','due_date','redec_proba','redec_score','redec_bin']]
df0["due_date"] = df0["due_date"].astype('datetime64[ns]')

In [18]:
ddf = ddf[['business_id','due_date','DPD_plus_15_next']]

In [22]:
dff = ddf.merge(df0,how='inner',left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])

In [23]:
dff.shape

(3079, 17)

In [24]:
dff.head()

,business_id,due_date,DPD_plus_15_next,emi_no,median_running_balance_2nd_m,median_amount_debited_2m,distinct_debit_txns_1m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_amt_drawn_mom,ratio_ach_debit_credit_freq_1m,ratio_amt_cashin_mom,ratio_revenue_1m_2m,distinct_cashins_1m_prev,redec_proba,redec_score,redec_bin
0,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-29,0.0,2,-0.559013,-0.554756,-0.537224,-1.195758,-1.726065,-0.275416,-0.732551,-0.347744,-0.156665,2.881640,0.578490,421.509705,2
1,939e1e8c-47d5-4600-b379-bf855be187f7,2023-05-04,0.0,2,-0.568955,-0.549359,-0.552790,-0.398808,0.853616,0.057691,-0.591285,-0.350400,-0.157183,-1.054347,0.468840,531.160143,2
2,5f46481b-1a69-43f7-9389-fb4887c17545,2023-05-04,0.0,2,-0.131378,4.807189,-0.179188,-0.343359,0.853616,1.390415,3.929247,-0.347017,-0.146627,-0.004751,0.005181,994.819278,5
3,4697e8dd-4b6a-4ee6-99d3-a8bbca6c1f8d,2023-05-04,0.0,2,1.419429,2.045391,-0.498307,-1.580253,0.853616,-0.275490,1.245181,-0.348842,-0.155337,-0.529549,0.016380,983.620092,5
4,ace8b476-54fe-4172-bf90-eb201d6b22af,2023-05-04,0.0,2,-0.083098,4.594640,-0.381556,-0.041520,0.438704,-0.275490,0.397582,-0.348948,-0.145762,-0.004751,0.015180,984.819536,5


In [25]:
# dff.to_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/test_dataset.pkl')

In [ ]:
pd.crosstab(dff['rs2_bin'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
pd.crosstab(dff['rs1_bin'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
pd.crosstab(dff['redec_bin'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
dff.loc[dff['DPD_plus_15_next']==1,'emi_no'].value_counts()

In [ ]:
pd.crosstab(dff['emi_no'],dff['DPD_plus_15_next']).to_clipboard()

In [ ]:
temp = dff[dff['emi_no'].isin([2,3])]

In [ ]:
temp.shape

In [ ]:
pd.crosstab(temp['rs2_bin'],temp['DPD_plus_15_next']).to_clipboard()

In [ ]:
dff['redec_bin'].value_counts()

In [ ]:
tdf = dff.groupby('redec_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf

In [ ]:
tdf = dff.groupby('rs2_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff.groupby('rs1_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff.groupby(['rs2_bin','redec_bin'],as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff[dff['emi_no'].isin([2,3])].groupby(['redec_bin','emi_no'],as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf.to_clipboard()

In [ ]:
tdf = dff.groupby(['redec_bin','DPD_plus_3'],as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['volume%'] = np.round(tdf['count']*100/len(dff),1)
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf

#### Bucketing on GA dataset

In [ ]:
dff.shape

In [ ]:
tdf = dff.groupby('rs2_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
tdf['default_rate'] = np.round(tdf['sum']*100/tdf['count'],1)
tdf

In [ ]:
ttdf = dff.groupby('redec_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
ttdf['default_rate'] = np.round(ttdf['sum']*100/ttdf['count'],1)
ttdf

In [ ]:
dff[(dff['redec_proba'] > 0.7)].shape[0]/len(dff)

In [ ]:
dff[(dff['redec_proba'] <= 0.1)].shape[0]/len(dff)

In [ ]:
dff[(dff['redec_proba'] > 0.35) & (dff['redec_proba'] <= 0.48)].shape[0]/len(dff)

In [ ]:
def new_buckets(x):
    if x > 0.6:
        return 1
    if x > 0.46:
        return 2
    if x > 0.31:
        return 3
    if x > 0.08:
        return 4
    return 5

In [ ]:
# def new_buckets(x):
#     if x > 0.7:
#         return 1
#     if x > 0.48:
#         return 2
#     if x > 0.35:
#         return 3
#     if x > 0.1:
#         return 4
#     return 5

In [ ]:
# def rs2_buckets(x):
#     if x > 0.7:
#         return 1
#     if x > 0.59:
#         return 2
#     if x > 0.46:
#         return 3
#     if x > 0.15:
#         return 4
#     return 5

In [ ]:
df0 = dff.copy()

In [ ]:
df0['rds_bin'] = df0['redec_proba'].apply(lambda x: new_buckets(x))

In [ ]:
ttdf = df0.groupby('rds_bin',as_index=False)['DPD_plus_15_next'].agg(['sum','count'])
ttdf['volume%'] = np.round(ttdf['count']*100/len(df0),1)
ttdf['default_rate'] = np.round(ttdf['sum']*100/ttdf['count'],1)
ttdf

In [ ]:
ttdf.to_clipboard()

#### Saving scored dataset

In [ ]:
df0.drop('redec_bin',axis=1,inplace=True)

In [ ]:
df0.shape

In [ ]:
df0.head()

In [ ]:
# df0.to_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/new_dataset_scored.pkl')

In [ ]:
# from sqlalchemy.types import NVARCHAR
# # from conf.config import SQLQuery
# q = SQLQuery('snowflake')

# df0.to_sql(name='rds1_GA_scored_dataset',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in df0})